In [116]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
import os
import dill
import json
from pathlib import Path
from typing import Any

import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from jax.tree_util import tree_flatten, tree_unflatten, tree_map

import sys
from trajectronpp import visualization
from trajectronpp.model.trajectron import Trajectron
from trajectronpp.environment.node_type import NodeType
from nuscenes.map_expansion.map_api import NuScenesMap

import helper # local file

## Load nuScenes SDK and data

In [118]:
nuScenes_data_path = Path("~/datasets/nuscenes").expanduser() # Data Path to nuScenes data set 
nuScenes_devkit_path = './devkit/python-sdk/'
sys.path.append(nuScenes_devkit_path)
nusc_map = NuScenesMap(dataroot=nuScenes_data_path, map_name='boston-seaport')

In [119]:
line_colors = ['#375397','#80CBE5','#ABCB51','#F05F78', '#C8B0B0']

# Map Encoding Demo

In [120]:
#with open('../processed/nuScenes_test_full.pkl', 'rb') as f:
#    eval_env = dill.load(f, encoding='latin1')
data_path = Path("~/datasets/trajectronpp2").expanduser()
path = data_path / "nuScenes_train_full.pkl"
eval_env = dill.loads(path.read_bytes(), encoding='latin1')
eval_scenes = eval_env.scenes

In [121]:
ph = 6
model_dir = data_path / "models" / "model_final"
eval_stg, hyp = helper.load_model(model_dir, eval_env, ts=12)


Loading from /home/rdyro/datasets/trajectronpp2/models/model_final/model_registrar-12.pt
Loaded!



In [122]:
scene = eval_scenes[25]
scene.name

'34'

In [123]:
# Define ROI in nuScenes Map
x_min = 773.0
x_max = 1100.0
y_min = 1231.0
y_max = 1510.0

In [9]:
layers = ['drivable_area',
          'road_segment',
          'lane',
          'ped_crossing',
          'walkway',
          'stop_line',
          'road_divider',
          'lane_divider']

In [10]:
with torch.no_grad():
    timestep = np.array([2])
    predictions = eval_stg.predict(scene, timestep, ph, num_samples=500)

    predictions_mm = eval_stg.predict(
        scene, timestep, ph, num_samples=1, z_mode=True, gmm_mode=True
    )

/home/rdyro/.pyenv/versions/devel/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [112]:
def set_attr(obj: Any, key: str, value: Any, delim: str = ".") -> None:
    if delim in key:
        attr, sub_key = key.split(delim, 1)
        set_attr(getattr(obj, attr), sub_key, value, delim)
    else:
        if hasattr(obj, key):
            delattr(obj, key)
        setattr(obj, key, value)


def replace_params(
    trajectronpp_model: Trajectron, new_params: dict[str, dict[str, dict[str, Any]]]
) -> None:
    for model, model_params in zip(trajectronpp_model.node_models_dict.values(), new_params):
        for node, new_params in zip(model.node_modules.values(), model_params):
            for param_name, new_param in new_params.items():
                set_attr(node, param_name, new_param)


def replace_buffers(
    trajectronpp_model: Trajectron, new_buffers: dict[str, dict[str, dict[str, Any]]]
) -> None:
    for model, model_params in zip(trajectronpp_model.node_models_dict.values(), new_buffers):
        for node, new_buffers in zip(model.node_modules.values(), model_params):
            for param_name, new_param in new_buffers.items():
                set_attr(node, param_name, new_param)


def obtain_params(trajectronpp_model: Trajectron) -> dict[str, dict[str, dict[str, Any]]]:
    params = [
        [dict(node.named_parameters()) for k, node in model.node_modules.items()]
        for (model_key, model) in trajectronpp_model.node_models_dict.items()
    ]
    buffers = [
        [dict(node.named_buffers()) for k, node in model.node_modules.items()]
        for (model_key, model) in trajectronpp_model.node_models_dict.items()
    ]
    return params, buffers

In [126]:
new_params, new_buffers = obtain_params(eval_stg)

In [147]:
eval_stg.eval_loss

<bound method Trajectron.eval_loss of <trajectronpp.model.trajectron.Trajectron object at 0x7f1f62597040>>

In [135]:
p = torch.cat([x.reshape(-1) for x in tree_flatten(new_params)[0]])

In [141]:
int(p.shape[0]) / 1e6

0.268496

In [106]:
replace_params(eval_stg, tree_map(lambda x: torch.tensor(x, requires_grad=False), new_buffers))
#with torch.no_grad():
timestep = np.array([2])
predictions = eval_stg.predict(scene, timestep, ph, num_samples=500)

predictions_mm = eval_stg.predict(
    scene, timestep, ph, num_samples=1, z_mode=True, gmm_mode=True
)

In [146]:
scene.get_scene_graph(0)

TypeError: object of type 'NoneType' has no len()

In [107]:
predictions_mm

{2: {VEHICLE/8c59ef6440c6436dab3a1917b1f0fc19: array([[[[106.096016, 235.08623 ],
           [106.05085 , 236.85002 ],
           [105.96373 , 238.59822 ],
           [105.83794 , 240.30408 ],
           [105.6819  , 241.96602 ],
           [105.50419 , 243.60284 ]]]], dtype=float32),
  VEHICLE/ego: array([[[[100.37622 , 172.79352 ],
           [ 98.08934 , 168.9745  ],
           [ 95.731186, 165.1945  ],
           [ 93.35069 , 161.42601 ],
           [ 90.96983 , 157.65694 ],
           [ 88.57962 , 153.89436 ]]]], dtype=float32)}}

In [102]:
list(list(list(eval_stg.node_models_dict.values())[0].node_modules.values())[0].parameters())

[]

In [ ]:
eval_stg()

In [62]:
new_new_params = tree_map(lambda x: torch.tensor(x.data, requires_grad=True), new_buffers)

/tmp/ipykernel_344548/686976958.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_new_params = tree_map(lambda x: torch.tensor(x.data, requires_grad=True), new_params)


In [64]:
replace_params(eval_stg, new_new_params)

TypeError: cannot assign 'torch.FloatTensor' as parameter 'bias_hh_l0' (torch.nn.Parameter or None expected)

In [60]:
list(list(list(eval_stg.node_models_dict.values())[0].node_modules.values())[2].named_parameters())

[('weight',
  Parameter containing:
  tensor([[ 1.8483e-01,  8.1790e-02,  1.1442e+00,  2.4283e-01, -1.3534e-01,
           -1.7418e-02, -3.6565e-01, -1.6700e+00],
          [-3.4892e-01,  2.3717e-01, -1.5227e+00, -3.4297e-01, -2.6075e-01,
           -5.2254e-01,  4.0523e-01,  1.1121e+00],
          [-6.7042e-02, -6.8654e-02, -8.1052e-01,  5.3605e-01,  6.5733e-01,
           -2.4727e-01, -4.1067e-01,  3.4191e+00],
          [-2.7230e-01, -2.9231e-01,  8.2400e-01, -9.5478e-01,  3.7015e-01,
            4.2478e-03,  1.2588e-01, -2.3870e+00],
          [ 3.3160e-01,  6.0227e-02,  5.3238e-01, -3.4920e-01,  3.1534e-01,
           -6.3689e-02, -1.5377e-02, -1.6158e+00],
          [ 3.3606e-01,  1.5721e-01,  2.6032e-01, -6.0765e-01, -3.8763e-01,
            8.8488e-01, -1.1195e-01, -7.4491e-01],
          [-2.4717e-01,  2.9009e-01,  1.3242e+00,  9.5324e-01,  2.2291e-01,
            1.4289e-01, -4.2244e-01, -2.6579e+00],
          [-3.5038e-02, -4.2763e-02,  2.9796e-01,  2.6080e-01, -1.2312e-01,

In [72]:
from utils_nn_func import generate_fwd_fn

In [73]:
m = torch.nn.Linear(64, 64)
w = m.weight
w2 = torch.randn((64, 64), requires_grad=True)

In [87]:
fwd_fn, params = generate_fwd_fn(m)

In [88]:
torch.autograd.grad(torch.sum(fwd_fn((w2, m.bias.detach()), torch.randn((1, 64)))), w2)

(tensor([[ 0.6563,  2.1941, -0.5043,  ..., -0.1099,  1.7977, -0.1752],
         [ 0.6563,  2.1941, -0.5043,  ..., -0.1099,  1.7977, -0.1752],
         [ 0.6563,  2.1941, -0.5043,  ..., -0.1099,  1.7977, -0.1752],
         ...,
         [ 0.6563,  2.1941, -0.5043,  ..., -0.1099,  1.7977, -0.1752],
         [ 0.6563,  2.1941, -0.5043,  ..., -0.1099,  1.7977, -0.1752],
         [ 0.6563,  2.1941, -0.5043,  ..., -0.1099,  1.7977, -0.1752]]),)

In [76]:
params

(tensor([[ 0.0648,  0.0662,  0.0354,  ..., -0.0759,  0.0125,  0.0530],
         [ 0.1013, -0.0934, -0.1132,  ..., -0.1039,  0.1028, -0.0741],
         [-0.1187, -0.0496,  0.0950,  ...,  0.0719, -0.0761, -0.0425],
         ...,
         [-0.0394, -0.0084, -0.0867,  ...,  0.1052, -0.0178, -0.1102],
         [ 0.0144, -0.1027, -0.0871,  ...,  0.0379,  0.1069, -0.1231],
         [ 0.1236,  0.0124,  0.1154,  ..., -0.0633,  0.1007,  0.1134]],
        requires_grad=True),
 tensor([ 0.0718, -0.0494,  0.0934, -0.0121, -0.0431, -0.0014,  0.0871, -0.0234,
          0.1105,  0.1100,  0.0757, -0.0488, -0.0533, -0.0752, -0.0175, -0.0522,
          0.0168, -0.0668,  0.0199,  0.0640,  0.0457, -0.0177, -0.0782, -0.1119,
         -0.0571, -0.0931,  0.0091,  0.0128, -0.0550, -0.0465, -0.1075, -0.0553,
          0.0518,  0.0572,  0.0008, -0.0236,  0.0119,  0.0506, -0.0859, -0.0319,
          0.0171, -0.0764, -0.1045,  0.0057, -0.1090, -0.0673, -0.1095,  0.0631,
         -0.0109,  0.1026, -0.1104,  0.1112,

In [70]:
m.weight = None

In [71]:
m.weight = w2

TypeError: cannot assign 'torch.FloatTensor' as parameter 'weight' (torch.nn.Parameter or None expected)

In [49]:
obtain_params(eval_stg)

[[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}],
 [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]]

In [36]:
type(next(new_buffers.values()))

TypeError: 'dict_values' object is not an iterator

In [37]:
tree_map(lambda x: x.shape, new_buffers)

TypeError: '<' not supported between instances of 'NodeType' and 'NodeType'

In [34]:
new_buffers = obtain_params(eval_stg)
replace(eval_stg, tree_map(lambda x: None, new_buffers))

TypeError: '<' not supported between instances of 'NodeType' and 'NodeType'

In [43]:
flat_params, tree_struct = tree_flatten(new_buffers)

In [59]:
new_new_params = tree_map(lambda x: torch.nn.Parameter(5 * torch.tensor(x)), new_buffers)
none_params = tree_map(lambda x: None, new_buffers)

/tmp/ipykernel_226400/1096791999.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_new_params = tree_map(lambda x: torch.nn.Parameter(5 * torch.tensor(x)), new_params)


In [60]:
replace(eval_stg.node_models_dict.values(), none_params)

In [61]:
replace(eval_stg.node_models_dict.values(), new_new_params)

In [25]:
set(p.dtype for p in all_params)

{torch.float32}

In [27]:
vehicle_node = [k for k in eval_stg.node_models_dict.keys() if k.name == "VEHICLE"][0]

In [36]:
sum(p.numel() for p in list(eval_stg.node_models_dict[vehicle_node].node_modules.values())[0].parameters())

5376

In [12]:
predictions

{2: {VEHICLE/8c59ef6440c6436dab3a1917b1f0fc19: array([[[[106.03945 , 235.0936  ],
           [105.841705, 236.90077 ],
           [105.6286  , 238.95224 ],
           [105.29999 , 241.18375 ],
           [104.94821 , 243.65823 ],
           [104.63774 , 246.43211 ]],
  
          [[105.96816 , 235.21712 ],
           [105.650925, 237.35799 ],
           [105.21167 , 239.80815 ],
           [104.55566 , 242.57571 ],
           [103.964424, 244.68466 ],
           [103.37962 , 246.03876 ]],
  
          [[106.112686, 235.22942 ],
           [106.018196, 237.34065 ],
           [105.595764, 239.56827 ],
           [104.98506 , 242.01279 ],
           [104.19098 , 244.66614 ],
           [103.40302 , 247.46817 ]],
  
          ...,
  
          [[105.65614 , 235.09387 ],
           [104.768234, 236.79953 ],
           [103.7497  , 238.71715 ],
           [102.30269 , 240.90854 ],
           [100.605606, 243.30559 ],
           [ 98.747986, 245.87991 ]],
  
          [[106.056305, 234.93816

## Prediction including Map Encoding

In [ ]:
ph = 6
with torch.no_grad():
    timestep = np.array([2])
    predictions = eval_stg.predict(scene, timestep, ph, num_samples=500)

    predictions_mm = eval_stg.predict(
        scene, timestep, ph, num_samples=1, z_mode=True, gmm_mode=True
    )

    # Plot predicted timestep for random scene in map
    my_patch = (x_min, y_min, x_max, y_max)
    fig, ax = nusc_map.render_map_patch(
        my_patch, layers, figsize=(10, 10), alpha=0.1, render_egoposes_range=False
    )

    ax.plot([], [], "ko-", zorder=620, markersize=4, linewidth=2, alpha=0.7, label="Ours (MM)")

    ax.plot(
        [],
        [],
        "w--o",
        label="Ground Truth",
        linewidth=3,
        path_effects=[pe.Stroke(linewidth=4, foreground="k"), pe.Normal()],
    )

    helper.plot_vehicle_nice(
        ax, predictions, scene.dt, max_hl=10, ph=ph, map=None, x_min=x_min, y_min=y_min
    )

    helper.plot_vehicle_mm(
        ax, predictions_mm, scene.dt, max_hl=10, ph=ph, map=None, x_min=x_min, y_min=y_min
    )

    ax.set_ylim((1385, 1435))
    ax.set_xlim((850, 900))
    leg = ax.legend(loc="upper right", fontsize=20, frameon=True)
    #ax.axis("off")
    for lh in leg.legendHandles:
        lh.set_alpha(0.5)
    ax.get_legend().remove()
    fig.show()
    fig.savefig("plots/qual_nuScenes_map_pos.pdf", dpi=300, bbox_inches="tight")

## Prediction without Map Encoding

In [ ]:
model_dir = os.path.join(log_dir, 'int_ee') 
eval_stg_nm, hyp = load_model(model_dir, eval_env, ts=12)

In [ ]:
ph = 6
with torch.no_grad():
    timestep = np.array([2])
    predictions = eval_stg_nm.predict(scene,
                                   timestep,
                                   ph,
                                   num_samples=500)

    predictions_mm = eval_stg_nm.predict(scene,
                                      timestep,
                                      ph,
                                      num_samples=1,
                                      z_mode=True,
                                      gmm_mode=True)

    # Plot predicted timestep for random scene in map
    my_patch = (x_min, y_min, x_max, y_max)
    fig, ax = nusc_map.render_map_patch(my_patch, layers, figsize=(10, 10), alpha=0.1, render_egoposes_range=False)

    ax.plot([], [], 'ko-',
            zorder=620,
            markersize=4,
            linewidth=2, alpha=0.7, label='Ours (MM)')

    ax.plot([],
            [],
            'w--o', label='Ground Truth',
            linewidth=3,
            path_effects=[pe.Stroke(linewidth=4, foreground='k'), pe.Normal()])

    plot_vehicle_nice(ax,
                      predictions,
                      scene.dt,
                      max_hl=10,
                      ph=ph,
                      map=None, x_min=x_min, y_min=y_min)

    plot_vehicle_mm(ax,
                    predictions_mm,
                    scene.dt,
                    max_hl=10,
                    ph=ph,
                    map=None, x_min=x_min, y_min=y_min)

    ax.set_ylim((1385, 1435))
    ax.set_xlim((850, 900))
    leg = ax.legend(loc='upper right', fontsize=20, frameon=True)
    ax.axis('off')
    for lh in leg.legendHandles:
        lh.set_alpha(.5)
    ax.get_legend().remove()
    fig.show()
    fig.savefig('plots/qual_nuScenes_no_map_pos.pdf', dpi=300, bbox_inches='tight')

## Prediction using velocity output

In [ ]:
model_dir = os.path.join(log_dir, 'ee_vel') 
eval_stg_vel, hyp = load_model(model_dir, eval_env, ts=12)

In [ ]:
ph = 6
with torch.no_grad():
    timestep = np.array([2])
    predictions = eval_stg_vel.predict(scene,
                                   timestep,
                                   ph,
                                   num_samples=500)

    predictions_mm = eval_stg_vel.predict(scene,
                                      timestep,
                                      ph,
                                      num_samples=1,
                                      z_mode=True,
                                      gmm_mode=True)

    # Plot predicted timestep for random scene in map
    my_patch = (x_min, y_min, x_max, y_max)
    fig, ax = nusc_map.render_map_patch(my_patch, layers, figsize=(10, 10), alpha=0.1, render_egoposes_range=False)

    ax.plot([], [], 'ko-',
            zorder=620,
            markersize=4,
            linewidth=2, alpha=0.7, label='Ours (MM)')

    ax.plot([],
            [],
            'w--o', label='Ground Truth',
            linewidth=3,
            path_effects=[pe.Stroke(linewidth=4, foreground='k'), pe.Normal()])

    plot_vehicle_nice(ax,
                      predictions,
                      scene.dt,
                      max_hl=10,
                      ph=ph,
                      map=None, x_min=x_min, y_min=y_min)

    plot_vehicle_mm(ax,
                    predictions_mm,
                    scene.dt,
                    max_hl=10,
                    ph=ph,
                    map=None, x_min=x_min, y_min=y_min)

    ax.set_ylim((1385, 1435))
    ax.set_xlim((850, 900))
    leg = ax.legend(loc='upper right', fontsize=20, frameon=True)
    ax.axis('off')
    for lh in leg.legendHandles:
        lh.set_alpha(.5)
    ax.get_legend().remove()
    fig.show()
    fig.savefig('plots/qual_nuScenes_no_map_vel.pdf', dpi=300, bbox_inches='tight')

## Prediction using velocity output and map

In [ ]:
model_dir = os.path.join(log_dir, 'me_vel') 
eval_stg_vel_map, hyp = load_model(model_dir, eval_env, ts=12)

In [ ]:
ph = 6
with torch.no_grad():
    timestep = np.array([2])
    predictions = eval_stg_vel_map.predict(scene,
                                   timestep,
                                   ph,
                                   num_samples=500)

    predictions_mm = eval_stg_vel_map.predict(scene,
                                      timestep,
                                      ph,
                                      num_samples=1,
                                      z_mode=True,
                                      gmm_mode=True)

    # Plot predicted timestep for random scene in map
    my_patch = (x_min, y_min, x_max, y_max)
    fig, ax = nusc_map.render_map_patch(my_patch, layers, figsize=(10, 10), alpha=0.1, render_egoposes_range=False)

    ax.plot([], [], 'ko-',
            zorder=620,
            markersize=4,
            linewidth=2, alpha=0.7, label='Ours (MM)')

    ax.plot([],
            [],
            'w--o', label='Ground Truth',
            linewidth=3,
            path_effects=[pe.Stroke(linewidth=4, foreground='k'), pe.Normal()])

    plot_vehicle_nice(ax,
                      predictions,
                      scene.dt,
                      max_hl=10,
                      ph=ph,
                      map=None, x_min=x_min, y_min=y_min)

    plot_vehicle_mm(ax,
                    predictions_mm,
                    scene.dt,
                    max_hl=10,
                    ph=ph,
                    map=None, x_min=x_min, y_min=y_min)

    ax.set_ylim((1385, 1435))
    ax.set_xlim((850, 900))
    leg = ax.legend(loc='upper right', fontsize=20, frameon=True)
    ax.axis('off')
    for lh in leg.legendHandles:
        lh.set_alpha(.5)
    ax.get_legend().remove()
    fig.show()
    fig.savefig('plots/qual_nuScenes_map_vel.pdf', dpi=300, bbox_inches='tight')

In [ ]:
my_patch = (0, 0, 1, 1)
fig, ax = nusc_map.render_map_patch(my_patch, layers, figsize=(1, 1), alpha=0.1, render_egoposes_range=False)
ax.plot([], [], 'ko',
            zorder=620,
            markersize=4,
            linewidth=2, alpha=0.7, label='Ours (ML)')

ax.plot([],
        [],
        'w--o', label='Ground Truth',
        linewidth=3,
        path_effects=[pe.Stroke(linewidth=4, foreground='k'), pe.Normal()])
leg = ax.legend(loc='upper left', fontsize=30, frameon=True)
for lh in leg.legendHandles:
    lh.set_alpha(.5)
ax.axis('off')
ax.grid('off')
fig.savefig('plots/qual_nuScenes_legend.pdf', dpi=300, bbox_inches='tight')